In [1]:
import pandas as pd
from haversine import haversine, Unit
import time
import numpy as np

In [2]:
# Load the data
properties = pd.read_csv('../data/pp-complete.csv')
properties

,{F887F88E-7D15-4415-804E-52EAC2F10958},70000,1995-07-07 00:00,MK15 9HP,D,N,F,31,Unnamed: 8,ALDRICH DRIVE,WILLEN,MILTON KEYNES,MILTON KEYNES.1,MILTON KEYNES.2,A,A.1
0,{40FD4DF2-5362-407C-92BC-566E2CCE89E9},44500,1995-02-03 00:00,SR6 0AQ,T,N,F,50,NaN,HOWICK PARK,SUNDERLAND,SUNDERLAND,SUNDERLAND,TYNE AND WEAR,A,A
1,{7A99F89E-7D81-4E45-ABD5-566E49A045EA},56500,1995-01-13 00:00,CO6 1SQ,T,N,F,19,NaN,BRICK KILN CLOSE,COGGESHALL,COLCHESTER,BRAINTREE,ESSEX,A,A
2,{28225260-E61C-4E57-8B56-566E5285B1C1},58000,1995-07-28 00:00,B90 4TG,T,N,F,37,NaN,RAINSBROOK DRIVE,SHIRLEY,SOLIHULL,SOLIHULL,WEST MIDLANDS,A,A
3,{444D34D7-9BA6-43A7-B695-4F48980E0176},51000,1995-06-28 00:00,DY5 1SA,S,N,F,59,NaN,MERRY HILL,BRIERLEY HILL,BRIERLEY HILL,DUDLEY,WEST MIDLANDS,A,A
4,{AE76CAF1-F8CC-43F9-8F63-4F48A2857D41},17000,1995-03-10 00:00,S65 1QJ,T,N,L,22,NaN,DENMAN STREET,ROTHERHAM,ROTHERHAM,ROTHERHAM,SOUTH YORKSHIRE,A,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28276222,{F3B6C198-17C3-6E40-E053-6C04A8C0B3B4},490000,2023-01-13 00:00,RM1 4SR,T,N,F,12,NaN,HEATHER GLEN,NaN,ROMFORD,HAVERING,GREATER LONDON,A,A
28276223,{F3B6C198-17C9-6E40-E053-6C04A8C0B3B4},365000,2023-01-10 00:00,SS2 4UA,F,N,L,"CENTENARY PLACE, 1",FLAT 22,SOUTHCHURCH BOULEVARD,NaN,SOUTHEND-ON-SEA,SOUTHEND-ON-SEA,SOUTHEND-ON-SEA,A,A
28276224,{F3B6C198-17D8-6E40-E053-6C04A8C0B3B4},730000,2023-01-05 00:00,CO13 0EY,S,N,F,HIGH VIEW,NaN,LITTLE CLACTON ROAD,GREAT HOLLAND,FRINTON-ON-SEA,TENDRING,ESSEX,A,A
28276225,{F3B6C198-1807-6E40-E053-6C04A8C0B3B4},379000,2023-01-06 00:00,CO15 5NA,D,N,F,12,NaN,DULWICH ROAD,HOLLAND-ON-SEA,CLACTON-ON-SEA,TENDRING,ESSEX,A,A


In [3]:
column_names = ["Transaction unique identifier", "Price", "Date of Transfer", "Postcode", "Property_Type", "Old_New", "Duration", "PAON", "SAON", "Street", "Locality", "Town_City", "District", "County", "PPD_Category_Type", "Record_Status"]
properties.columns = column_names
properties

,Transaction unique identifier,Price,Date of Transfer,Postcode,Property_Type,Old_New,Duration,PAON,SAON,Street,Locality,Town_City,District,County,PPD_Category_Type,Record_Status
0,{40FD4DF2-5362-407C-92BC-566E2CCE89E9},44500,1995-02-03 00:00,SR6 0AQ,T,N,F,50,NaN,HOWICK PARK,SUNDERLAND,SUNDERLAND,SUNDERLAND,TYNE AND WEAR,A,A
1,{7A99F89E-7D81-4E45-ABD5-566E49A045EA},56500,1995-01-13 00:00,CO6 1SQ,T,N,F,19,NaN,BRICK KILN CLOSE,COGGESHALL,COLCHESTER,BRAINTREE,ESSEX,A,A
2,{28225260-E61C-4E57-8B56-566E5285B1C1},58000,1995-07-28 00:00,B90 4TG,T,N,F,37,NaN,RAINSBROOK DRIVE,SHIRLEY,SOLIHULL,SOLIHULL,WEST MIDLANDS,A,A
3,{444D34D7-9BA6-43A7-B695-4F48980E0176},51000,1995-06-28 00:00,DY5 1SA,S,N,F,59,NaN,MERRY HILL,BRIERLEY HILL,BRIERLEY HILL,DUDLEY,WEST MIDLANDS,A,A
4,{AE76CAF1-F8CC-43F9-8F63-4F48A2857D41},17000,1995-03-10 00:00,S65 1QJ,T,N,L,22,NaN,DENMAN STREET,ROTHERHAM,ROTHERHAM,ROTHERHAM,SOUTH YORKSHIRE,A,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28276222,{F3B6C198-17C3-6E40-E053-6C04A8C0B3B4},490000,2023-01-13 00:00,RM1 4SR,T,N,F,12,NaN,HEATHER GLEN,NaN,ROMFORD,HAVERING,GREATER LONDON,A,A
28276223,{F3B6C198-17C9-6E40-E053-6C04A8C0B3B4},365000,2023-01-10 00:00,SS2 4UA,F,N,L,"CENTENARY PLACE, 1",FLAT 22,SOUTHCHURCH BOULEVARD,NaN,SOUTHEND-ON-SEA,SOUTHEND-ON-SEA,SOUTHEND-ON-SEA,A,A
28276224,{F3B6C198-17D8-6E40-E053-6C04A8C0B3B4},730000,2023-01-05 00:00,CO13 0EY,S,N,F,HIGH VIEW,NaN,LITTLE CLACTON ROAD,GREAT HOLLAND,FRINTON-ON-SEA,TENDRING,ESSEX,A,A
28276225,{F3B6C198-1807-6E40-E053-6C04A8C0B3B4},379000,2023-01-06 00:00,CO15 5NA,D,N,F,12,NaN,DULWICH ROAD,HOLLAND-ON-SEA,CLACTON-ON-SEA,TENDRING,ESSEX,A,A


In [4]:
# Convert 'Date of Transfer' to datetime if it's not already
properties['Date of Transfer'] = pd.to_datetime(properties['Date of Transfer'])

# Filter out data of 2023
properties = properties[properties['Date of Transfer'].dt.year < 2023]
properties

,Transaction unique identifier,Price,Date of Transfer,Postcode,Property_Type,Old_New,Duration,PAON,SAON,Street,Locality,Town_City,District,County,PPD_Category_Type,Record_Status
0,{40FD4DF2-5362-407C-92BC-566E2CCE89E9},44500,1995-02-03,SR6 0AQ,T,N,F,50,NaN,HOWICK PARK,SUNDERLAND,SUNDERLAND,SUNDERLAND,TYNE AND WEAR,A,A
1,{7A99F89E-7D81-4E45-ABD5-566E49A045EA},56500,1995-01-13,CO6 1SQ,T,N,F,19,NaN,BRICK KILN CLOSE,COGGESHALL,COLCHESTER,BRAINTREE,ESSEX,A,A
2,{28225260-E61C-4E57-8B56-566E5285B1C1},58000,1995-07-28,B90 4TG,T,N,F,37,NaN,RAINSBROOK DRIVE,SHIRLEY,SOLIHULL,SOLIHULL,WEST MIDLANDS,A,A
3,{444D34D7-9BA6-43A7-B695-4F48980E0176},51000,1995-06-28,DY5 1SA,S,N,F,59,NaN,MERRY HILL,BRIERLEY HILL,BRIERLEY HILL,DUDLEY,WEST MIDLANDS,A,A
4,{AE76CAF1-F8CC-43F9-8F63-4F48A2857D41},17000,1995-03-10,S65 1QJ,T,N,L,22,NaN,DENMAN STREET,ROTHERHAM,ROTHERHAM,ROTHERHAM,SOUTH YORKSHIRE,A,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28147319,{E2D14905-969C-4C2D-E053-6B04A8C0422B},475000,2022-05-27,ME19 4SA,S,N,F,31,NaN,SAXON CLOSE,KINGS HILL,WEST MALLING,TONBRIDGE AND MALLING,KENT,A,A
28147320,{E2D14905-969D-4C2D-E053-6B04A8C0422B},365200,2022-03-01,DA9 9NN,S,N,F,2,NaN,HIGH STREET,NaN,GREENHITHE,DARTFORD,KENT,B,A
28147321,{E2D14905-969E-4C2D-E053-6B04A8C0422B},243000,2022-03-17,TN24 9JP,T,N,F,18,NaN,NINE ACRES,KENNINGTON,ASHFORD,ASHFORD,KENT,A,A
28147322,{E2D14905-969F-4C2D-E053-6B04A8C0422B},377000,2022-04-22,TN28 8XH,S,N,F,19,NaN,ELLIS DRIVE,NaN,NEW ROMNEY,FOLKESTONE AND HYTHE,KENT,A,A


In [5]:
missing_postcodes = properties['Postcode'].isna().sum()
print(f"Number of missing postcodes: {missing_postcodes}")

Number of missing postcodes: 45834


In [6]:
# List of postcode prefixes for Wales
wales_postcode_start = ['LL', 'NP', 'CF', 'SA', 'CH', 'SY', 'HR', 'LD']

# Create a boolean mask
mask = properties['Postcode'].str[:2].isin(wales_postcode_start)

# Invert the mask, to get only rows that are not in Wales
properties = properties[~mask]

# Checking the result
properties

,Transaction unique identifier,Price,Date of Transfer,Postcode,Property_Type,Old_New,Duration,PAON,SAON,Street,Locality,Town_City,District,County,PPD_Category_Type,Record_Status
0,{40FD4DF2-5362-407C-92BC-566E2CCE89E9},44500,1995-02-03,SR6 0AQ,T,N,F,50,NaN,HOWICK PARK,SUNDERLAND,SUNDERLAND,SUNDERLAND,TYNE AND WEAR,A,A
1,{7A99F89E-7D81-4E45-ABD5-566E49A045EA},56500,1995-01-13,CO6 1SQ,T,N,F,19,NaN,BRICK KILN CLOSE,COGGESHALL,COLCHESTER,BRAINTREE,ESSEX,A,A
2,{28225260-E61C-4E57-8B56-566E5285B1C1},58000,1995-07-28,B90 4TG,T,N,F,37,NaN,RAINSBROOK DRIVE,SHIRLEY,SOLIHULL,SOLIHULL,WEST MIDLANDS,A,A
3,{444D34D7-9BA6-43A7-B695-4F48980E0176},51000,1995-06-28,DY5 1SA,S,N,F,59,NaN,MERRY HILL,BRIERLEY HILL,BRIERLEY HILL,DUDLEY,WEST MIDLANDS,A,A
4,{AE76CAF1-F8CC-43F9-8F63-4F48A2857D41},17000,1995-03-10,S65 1QJ,T,N,L,22,NaN,DENMAN STREET,ROTHERHAM,ROTHERHAM,ROTHERHAM,SOUTH YORKSHIRE,A,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28147319,{E2D14905-969C-4C2D-E053-6B04A8C0422B},475000,2022-05-27,ME19 4SA,S,N,F,31,NaN,SAXON CLOSE,KINGS HILL,WEST MALLING,TONBRIDGE AND MALLING,KENT,A,A
28147320,{E2D14905-969D-4C2D-E053-6B04A8C0422B},365200,2022-03-01,DA9 9NN,S,N,F,2,NaN,HIGH STREET,NaN,GREENHITHE,DARTFORD,KENT,B,A
28147321,{E2D14905-969E-4C2D-E053-6B04A8C0422B},243000,2022-03-17,TN24 9JP,T,N,F,18,NaN,NINE ACRES,KENNINGTON,ASHFORD,ASHFORD,KENT,A,A
28147322,{E2D14905-969F-4C2D-E053-6B04A8C0422B},377000,2022-04-22,TN28 8XH,S,N,F,19,NaN,ELLIS DRIVE,NaN,NEW ROMNEY,FOLKESTONE AND HYTHE,KENT,A,A


In [7]:
num_unique_postcodes = properties['Postcode'].nunique()
print(f'The number of unique postcodes is: {num_unique_postcodes}')


The number of unique postcodes is: 1193459


In [8]:
postcodes = pd.read_csv("../data/NSPL21_MAY_2023_UK.csv")
postcodes

/var/folders/nr/xgt7yplx50b7g0btpmjwcjf00000gn/T/ipykernel_768/4145548786.py:1: DtypeWarning: Columns (26,31) have mixed types. Specify dtype option on import or set low_memory=False.
  postcodes = pd.read_csv("../data/NSPL21_MAY_2023_UK.csv")


,pcd,pcd2,pcds,dointr,doterm,usertype,oseast1m,osnrth1m,osgrdind,oa21,...,bua22,ru11ind,oac11,lat,long,lep1,lep2,pfa,imd,icb
0,AB1 0AA,AB1 0AA,AB1 0AA,198001,199606.0,0,385386.0,801193.0,1,S00090303,...,S99999999,3.0,1C3,57.101474,-2.242851,S99999999,NaN,S23000009,6715,S99999999
1,AB1 0AB,AB1 0AB,AB1 0AB,198001,199606.0,0,385177.0,801314.0,1,S00090303,...,S99999999,3.0,1C3,57.102554,-2.246308,S99999999,NaN,S23000009,6715,S99999999
2,AB1 0AD,AB1 0AD,AB1 0AD,198001,199606.0,0,385053.0,801092.0,1,S00090399,...,S99999999,3.0,6A1,57.100556,-2.248342,S99999999,NaN,S23000009,6715,S99999999
3,AB1 0AE,AB1 0AE,AB1 0AE,199402,199606.0,0,384600.0,799300.0,8,S00091322,...,S99999999,6.0,1A2,57.084444,-2.255708,S99999999,NaN,S23000009,5069,S99999999
4,AB1 0AF,AB1 0AF,AB1 0AF,199012,199207.0,1,384460.0,800660.0,8,S00090299,...,S99999999,3.0,6A4,57.096656,-2.258102,S99999999,NaN,S23000009,6253,S99999999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2690846,ZE3 9JW,ZE3 9JW,ZE3 9JW,198001,NaN,0,438975.0,1110038.0,1,S00129022,...,S99999999,8,1B1,59.873651,-1.305697,S99999999,NaN,S23000009,4141,S99999999
2690847,ZE3 9JX,ZE3 9JX,ZE3 9JX,198001,NaN,0,438872.0,1110219.0,1,S00129022,...,S99999999,8,1B1,59.875286,-1.307502,S99999999,NaN,S23000009,4141,S99999999
2690848,ZE3 9JY,ZE3 9JY,ZE3 9JY,198001,NaN,0,438498.0,1112029.0,1,S00129025,...,S99999999,8,1A1,59.891572,-1.313847,S99999999,NaN,S23000009,4141,S99999999
2690849,ZE3 9JZ,ZE3 9JZ,ZE3 9JZ,198001,NaN,0,438662.0,1112122.0,1,S00129025,...,S99999999,8,1A1,59.892392,-1.310899,S99999999,NaN,S23000009,4141,S99999999


In [9]:
print((postcodes['pcd'] == postcodes['pcd2']).all())
print((postcodes['pcd'] == postcodes['pcds']).all())
print((postcodes['pcd2'] == postcodes['pcds']).all())

False
False
False


In [10]:
diff_pcd_pcd2 = postcodes[postcodes['pcd'] != postcodes['pcd2']]
diff_pcd_pcds = postcodes[postcodes['pcd'] != postcodes['pcds']]
diff_pcd2_pcds = postcodes[postcodes['pcd2'] != postcodes['pcds']]

# Print a few rows from each dataframe
print(diff_pcd_pcd2.head())
print(diff_pcd_pcds.head())
print(diff_pcd2_pcds.head())


       pcd      pcd2     pcds  dointr    doterm  usertype  oseast1m  osnrth1m  \
0  AB1 0AA  AB1  0AA  AB1 0AA  198001  199606.0         0  385386.0  801193.0   
1  AB1 0AB  AB1  0AB  AB1 0AB  198001  199606.0         0  385177.0  801314.0   
2  AB1 0AD  AB1  0AD  AB1 0AD  198001  199606.0         0  385053.0  801092.0   
3  AB1 0AE  AB1  0AE  AB1 0AE  199402  199606.0         0  384600.0  799300.0   
4  AB1 0AF  AB1  0AF  AB1 0AF  199012  199207.0         1  384460.0  800660.0   

   osgrdind       oa21  ...      bua22 ru11ind oac11        lat      long  \
0         1  S00090303  ...  S99999999     3.0   1C3  57.101474 -2.242851   
1         1  S00090303  ...  S99999999     3.0   1C3  57.102554 -2.246308   
2         1  S00090399  ...  S99999999     3.0   6A1  57.100556 -2.248342   
3         8  S00091322  ...  S99999999     6.0   1A2  57.084444 -2.255708   
4         8  S00090299  ...  S99999999     3.0   6A4  57.096656 -2.258102   

        lep1 lep2        pfa   imd        icb  
0 

In [11]:
num_unique_postcodes = postcodes['pcd'].nunique()
print(f'The number of unique postcodes is: {num_unique_postcodes}')


The number of unique postcodes is: 2690851


In [12]:
# Get unique postcodes from both dataframes
unique_properties_postcodes = properties['Postcode'].unique()
unique_postcodes_postcodes = postcodes['pcd'].unique()



In [13]:
unique_postcodes_postcodes_2 = postcodes['pcd2'].unique()
unique_postcodes_postcodes_s = postcodes['pcds'].unique()

In [14]:
# Compare
common_postcodes = set(unique_properties_postcodes).intersection(set(unique_postcodes_postcodes_s))
print(f"Number of common postcodes: {len(common_postcodes)}")


Number of common postcodes: 1192278


In [15]:
common_postcodes

{'GL8 8HA',
 'SS2 4RJ',
 'DA14 6BQ',
 'KT6 6HR',
 'ME7 4EB',
 'DT4 0NB',
 'E17 6RR',
 'SM6 9QQ',
 'CW7 1DR',
 'TQ6 0JG',
 'SO22 6PZ',
 'LE10 1HL',
 'M22 4NY',
 'GU5 9ER',
 'TS18 5JG',
 'B21 8DW',
 'OL15 8JB',
 'DH5 0PH',
 'CT12 6EF',
 'SO16 4QS',
 'M3 5JL',
 'TR21 0NJ',
 'PO12 4UG',
 'ST3 2RA',
 'CV6 6FR',
 'TR8 5NX',
 'CB5 8SG',
 'BH24 1RT',
 'LE2 9RB',
 'YO26 4XZ',
 'PL35 0EA',
 'KT15 3NS',
 'TN12 9SX',
 'LA1 4ET',
 'BS16 7FN',
 'WS12 4SS',
 'CA7 3JZ',
 'DT6 6RD',
 'M21 8FE',
 'DE1 3RT',
 'PE9 2ST',
 'NR35 2TZ',
 'HP7 9BD',
 'ME9 7DD',
 'DE11 9TS',
 'YO22 4HT',
 'PL14 3QD',
 'SK10 2JJ',
 'BD6 2DD',
 'LS11 9NJ',
 'NG2 1QE',
 'OX14 1PA',
 'B60 2AR',
 'DY11 6SF',
 'RM5 3YD',
 'L10 4UT',
 'BD13 5EB',
 'SN15 5JR',
 'MK6 3DW',
 'TA2 8FL',
 'ME10 2UD',
 'PL2 1JH',
 'S44 6XU',
 'WA3 1AT',
 'SN3 3AU',
 'RM6 5UF',
 'PO15 5LD',
 'DN37 7BP',
 'N8 7BB',
 'ME5 8RJ',
 'CM3 1DJ',
 'CM3 4RZ',
 'SW6 2XD',
 'LS24 9TL',
 'SN12 7GF',
 'DA7 6HX',
 'NG16 3DZ',
 'BD11 2LL',
 'BH31 6HL',
 'GL53 0LG',
 'B62 9

In [16]:
# Create a DataFrame from the common_postcodes set
common_postcodes_df = pd.DataFrame(list(common_postcodes), columns=['Postcode'])

# Merge common_postcodes_df with the postcodes dataframe
df_final = pd.merge(common_postcodes_df, postcodes[['pcds', 'lat', 'long']], left_on='Postcode', right_on='pcds', how='left')

# Drop the redundant 'pcds' column
df_final.drop(columns=['pcds'], inplace=True)
df_final = df_final.rename(columns={"lat": "Lat", "long": "Long"})


In [17]:
df_final

,Postcode,Lat,Long
0,GL8 8HA,51.640620,-2.152272
1,SS2 4RJ,51.547842,0.739400
2,DA14 6BQ,51.422746,0.099688
3,KT6 6HR,51.393117,-0.299780
4,ME7 4EB,51.380130,0.557023
...,...,...,...
1192273,WS13 6SY,52.688117,-1.806688
1192274,LE17 6LB,52.454033,-1.057775
1192275,CB5 8FS,52.209282,0.136840
1192276,HD9 7BE,53.589470,-1.762828


In [18]:
# Specify the output file path for the merged CSV file
output_file_path = f'../data/Airport Distance/bq-unique-property-postcodes.csv'

# Write the merged_data DataFrame to a CSV file
df_final.to_csv(output_file_path, index=False)

In [19]:
# Select required columns from properties DataFrame
properties_selected = properties[['Price', 'Date of Transfer', 'Postcode']]

# Merge properties_selected with df_final
df_merged = pd.merge(properties_selected, df_final, on='Postcode', how='inner')
df_merged

,Price,Date of Transfer,Postcode,Lat,Long
0,44500,1995-02-03,SR6 0AQ,54.912539,-1.380977
1,42000,1995-06-23,SR6 0AQ,54.912539,-1.380977
2,31600,1996-04-19,SR6 0AQ,54.912539,-1.380977
3,35900,1997-09-02,SR6 0AQ,54.912539,-1.380977
4,39000,1998-04-27,SR6 0AQ,54.912539,-1.380977
...,...,...,...,...,...
26308204,335000,2022-04-08,SO15 0NE,50.921032,-1.467421
26308205,395000,2022-05-27,SO17 3TD,50.937320,-1.387404
26308206,117000,2022-01-21,NE4 6EQ,54.968487,-1.633696
26308207,615050,2022-04-27,CT15 4LD,51.224197,1.222978


In [20]:
df_merged = df_merged[['Price', 'Date of Transfer', 'Postcode']]
df_merged

,Price,Date of Transfer,Postcode
0,44500,1995-02-03,SR6 0AQ
1,42000,1995-06-23,SR6 0AQ
2,31600,1996-04-19,SR6 0AQ
3,35900,1997-09-02,SR6 0AQ
4,39000,1998-04-27,SR6 0AQ
...,...,...,...
26308204,335000,2022-04-08,SO15 0NE
26308205,395000,2022-05-27,SO17 3TD
26308206,117000,2022-01-21,NE4 6EQ
26308207,615050,2022-04-27,CT15 4LD


In [21]:
# Convert 'Year of Transfer' to datetime format
df_merged['Date of Transfer'] = pd.to_datetime(df_merged['Date of Transfer'])


# Filter rows where 'Year of Transfer' is before 22 February 2022
df_filtered = df_merged[df_merged['Date of Transfer'] < pd.Timestamp(2022, 2, 23)]

# Extract the year and assign it back to the 'Date of Transfer' column
df_filtered['Date of Transfer'] = df_filtered['Date of Transfer'].dt.year

df_filtered = df_filtered.rename(columns={'Date of Transfer': 'Year of Transfer'})
df_filtered

/var/folders/nr/xgt7yplx50b7g0btpmjwcjf00000gn/T/ipykernel_768/1732065156.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged['Date of Transfer'] = pd.to_datetime(df_merged['Date of Transfer'])
/var/folders/nr/xgt7yplx50b7g0btpmjwcjf00000gn/T/ipykernel_768/1732065156.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Date of Transfer'] = df_filtered['Date of Transfer'].dt.year


,Price,Year of Transfer,Postcode
0,44500,1995,SR6 0AQ
1,42000,1995,SR6 0AQ
2,31600,1996,SR6 0AQ
3,35900,1997,SR6 0AQ
4,39000,1998,SR6 0AQ
...,...,...,...
26308154,849999,2022,TN33 9LY
26308155,425000,2022,DT1 3HH
26308193,365000,2022,SS17 0AX
26308195,160000,2022,TS19 7QN


In [22]:
# Specify the output file path for the merged CSV file
output_file_path = f'../data/Airport Distance/bq-property_price_year_postcode.csv'

# Write the merged_data DataFrame to a CSV file
df_filtered.to_csv(output_file_path, index=False)